# Load packages and load the events dataframe

In [ ]:
import os
import sys
import os.path as op
sys.path.append('/home1/cjmac/code/general')
sys.path.append('/home1/cjmac/code/manning_replication')
sys.path.append('/home1/cjmac/code/projects/')
import data_io as dio
import matplotlib.pyplot as plt
import seaborn as sns
spike_value = 1
import warnings
warnings.filterwarnings( 'ignore' )
# warnings.filterwarnings( 'default' )

proj_dir = '/home1/cjmac/projects/time_cells'
output_f = os.path.join(proj_dir, 'analysis', 'events','U554_ses0-events_formatted.pkl')
events = dio.open_pickle(output_f)
import pandas as pd
import numpy as np

# Create an event list corresponding to gem placement

In [ ]:
# create dataframe by unnesting all item (key:value) pairs
events.sort_values(by=['time'])
all_value = pd.DataFrame( events.value.tolist() )
# get each trial's timeline info
timelines = list( all_value.loc[all_value.chosenTimelineItems.notna().values].chosenTimelineItems )
#
# timelines_placement = list( all_value.loc[all_value.keyCode.notna.values].keyCode )
#
events_recall = [ ( trial, chosengem ) for trial, gem in enumerate( timelines ) for chosengem in gem ]
#
events_recall = pd.DataFrame( events_recall, columns=['trial', 'chosengems_all' ] )
chosengems_all = pd.DataFrame( events_recall.chosengems_all.tolist() )
#
events_recall = pd.concat( [events_recall, chosengems_all ], axis = 1  )
#
events_recall = events_recall.drop( { 'chosengems_all', 'actualTime' }, axis = 1 )

events_recall.drop_duplicates(keep='last' )
events_recall['trial'] = events_recall['trial'] + 1

events_recall = events_recall.drop_duplicates(keep = 'last' )

events_recall = events_recall[ events_recall.chosenTime > 0 ]
#
events_recall['chosenTime'] = events_recall.groupby( 'trial' ).cumcount() + 1
events_recall[ 'trial_type' ] = 'recall'
events_recall = events_recall.rename( { 'trial':'list', 'name':'item', 'chosenTime':'position' }, axis= 'columns' )
events_recall = events_recall.iloc[:, [0, 3, 2, 1 ] ]
events_recall.insert(0, 'subject', 1 )
events_recall.insert(5 , 'category', 'gems' )
events_recall.insert(5 , 'list_type', 'pure'  )
#

In [ ]:
# events.loc[ all_value['keyCode'] == 'Mouse0', :].sort_values(by=['time']).to_csv('events_mouseclicks.csv')
events_recall

In [ ]:
all_value['isPressed'][ all_value['keyCode'] == 'Mouse0'] = np.cumsum(np.where(all_value['isPressed'][ all_value['keyCode'] == 'Mouse0'] == True, 1, -1))


In [ ]:
abs(all_value['isPressed'][ all_value['keyCode'] == 'Mouse0'])==2

# Create an event list corresponding to delay1 timestamps.

In [ ]:
tmp_events = events[ events['gameState'] == 'Delay1']
# print( tmp_events[0:100:1] )
tmp_times = events['time'][ events['gameState'] == 'Delay1']
# print( tmp_times )
tmp_diff = np.diff( tmp_times )
# print( type( tmp_diff ) )
delay1_start = tmp_times.iloc[ np.where( ( tmp_times - np.roll( tmp_times, +1 ) ) > 100000 )[0] ]
# print( type( np.where( ( tmp_times - np.roll( tmp_times, +1 ) ) > 100000 )[0] ) )
# print( np.where( ( tmp_times - np.roll( tmp_times, +1 ) ) > 100000 )[0] )
#print( delay1_start )
tmp_diff

# Create an event list corresponding to delay2 timestamps.

In [ ]:
tmp_events = events[ events['gameState'] == 'Delay2']
# print( tmp_events[0:100:1] )
tmp_times = events['time'][ events['gameState'] == 'Delay2']
# print( tmp_times )
tmp_diff = np.diff( tmp_times )
delay2_start = tmp_times.iloc[ np.where( ( tmp_times - np.roll( tmp_times, +1 ) ) > 100000 )[0] ]
#print( delay2_start )

# Load a neuron's spike data.

In [ ]:
import spike_preproc
flag_reload = 0
if flag_reload:
    reload( spike_preproc )

subj_sess = 'U554_ses0'


spikes = spike_preproc.format_spikes(subj_sess,
                                         conv_factor='infer',
                                         add_montage_info=False,
                                         fr_thresh=0.1,
                                         overwrite=True,
                                         save_output=True,
                                         split_files=True,
                                         verbose=False)

In [ ]:
spikes

In [ ]:
which_cell=23
trial=1

spike_times = np.subtract( spikes['spike_times'][which_cell], delay1_start.iloc[ trial ] )
spike_times = spike_times[(spike_times>=0 ) & (spike_times<=100000)]
# spike_times_binned[trial, [ spike_times ] ] = 1
spike_times    
    

# Create a PSTH + raster plot for Delay1 & Delay2

In [ ]:
which_cell = 23
import seaborn as sns

spike_times_binned = np.zeros((len(delay1_start),100000)) 

for trial in range(len(delay1_start)):
    #print( trial )
    spike_times = np.subtract( spikes['spike_times'][which_cell], delay1_start.iloc[ trial ] )
    spike_times = spike_times[(spike_times>=0 ) & (spike_times<=10000)]
    spike_times_binned[trial, [ spike_times ] ] = 1

In [ ]:
########################################
# draw raster into first subplot, axs[0]
# add shading for stimulus duration)
# axs[0].axvspan(light_onset_time, light_offset_time, alpha=0.5, color='greenyellow')

fig, axs = plt.subplots(2, 1, figsize=[10,5], sharex = True )

for trial in range(len(spike_times_binned)):
    spike_times = [i for i, x in enumerate(spike_times_binned[trial]) if x == spike_value]
    axs[0].vlines(spike_times, trial - 0.5, trial + 0.5, colors = 'black')
    
axs[0].set_xlim( [ 0, 10000 ] )  
axs[0].set_xticks( [ 0, 50000, 10000 ] )  
axs[0].tick_params(axis='y', labelsize=18, )
axs[0].set_title( 'Delay1', fontsize = 32 )
axs[0].set_ylabel( 'trial', fontsize = 24 )
print( axs[0].xaxis.get_visible())
#axs[0].xaxis.set_visible('False')
axs[0].spines['bottom'].set_visible(False)   


##
spike_times_binned_reshape = spike_times_binned.reshape( [230000, 10 ], order='F' )

spike_times_binned_reshape = np.sum( spike_times_binned_reshape, axis = 0 )
spike_rates_binned_reshape =(spike_times_binned_reshape/(25))

# Draw the PSTH
axs[1].bar(np.array( [500, 1500, 2500, 3500, 4500, 5500, 6500, 7500, 8500, 9500] ),
           spike_rates_binned_reshape, 10000, color = 'black')

axs[1].set_xlim( [ 0, 10000 ] )  
axs[1].set_xticks( [ 0, 5000, 10000 ] )  
axs[1].tick_params(axis='y', labelsize=18, )
axs[1].set_xticklabels( [ '0', '5', '10' ], fontsize = 20 )  
axs[1].set_ylabel( 'spikes/s', fontsize = 24 )
axs[1].set_xlabel( 'time (s)', fontsize = 24  )

plt.tight_layout()


plt.savefig('delay1.eps', dpi=300)


# Create a PSTH + raster plot for Delay2

spike_times_binned = np.zeros((len(delay2_start),100000)) 

for trial in range(len(delay2_start)):
    #print( trial )
    spike_times = np.subtract( spikes['spike_times'][which_cell], delay2_start.iloc[ trial ] )
    spike_times = spike_times[(spike_times>=0 ) & (spike_times<=10000)]
    spike_times_binned[trial, [ spike_times ] ] = 1

########################################
# draw raster into first subplot, axs[0]
# add shading for stimulus duration)
# axs[0].axvspan(light_onset_time, light_offset_time, alpha=0.5, color='greenyellow')

fig, axs = plt.subplots(2, 1, figsize=[10,5], sharex = True )

for trial in range(len(spike_times_binned)):
    spike_times = [i for i, x in enumerate(spike_times_binned[trial]) if x == spike_value]
    axs[0].vlines(spike_times, trial - 0.5, trial + 0.5, colors = 'black')

axs[0].set_xlim( [ 0, 100000 ] )  
axs[0].set_xticks( [ 0, 50000, 100000 ] )  
axs[0].tick_params(axis='y', labelsize=18, )
axs[0].set_title( 'Delay2', fontsize = 32 )
axs[0].set_ylabel( 'trial', fontsize = 24 )
axs[0].spines['bottom'].set_visible(False)
##
spike_times_binned_reshape = spike_times_binned.reshape( [250000, 10 ], order='F' )

spike_times_binned_reshape = np.sum( spike_times_binned_reshape, axis = 0 )
spike_rates_binned_reshape =(spike_times_binned_reshape/(25))

# Draw the PSTH
axs[1].bar(np.array( [500, 1500, 2500, 3500, 4500, 5500, 6500, 7500, 8500, 9500]),
           spike_rates_binned_reshape, 10000, color = 'black')

axs[1].set_xlim( [ 0, 100000 ] )  
axs[1].set_xticks( [ 0, 50000, 100000 ] )  
axs[1].tick_params(axis='y', labelsize=18, )
axs[1].set_xticklabels( [ '0', '5', '10' ], fontsize = 20 )  
axs[1].set_ylabel( 'spikes/s', fontsize = 24 )
axs[1].set_xlabel( 'time (s)', fontsize = 24  )


plt.tight_layout()

plt.savefig('delay2.eps', dpi=300)

In [ ]:
type( axs[0] )